# Moment propagation and first covering attempt

This notebook describes our attempt at using the moment propagation approach assuming that at each layer of the network, all propagated variables are Gaussians. Moreover, we make first experiments with propagating a covering. 

This notebook is also not relevant to our final result, except that it shows how the Gaussianity assumption is unrealistic, however it is included to document our work.

In [ ]:
include("../src/DSI.jl")
include("../src/Zono_utils.jl")
include("../src/PZono.jl")
include("../src/DSZ.jl")
include("../src/propagation.jl")

In [ ]:
using SpecialFunctions

# P-box setup

In [ ]:
using PyPlot
using Distributions

c = normal(interval(0,1),1)
ProbabilityBoundsAnalysis.plot(c)

arr = []
for mean in 0:0.01:1
    normal_dist = Normal(mean, 1)
    push!(arr, normal_dist)
    x = range(mean - 4, mean + 4, length=1000)
    cdf_values = cdf.(normal_dist, x)
    plot(x, cdf_values)
end

PyPlot.savefig("parametric.png")

In [ ]:
W1 = [1 -1.0; 1.0 1.]
b = [0.0; 0.0]
W2 = [1 -1.0; 1.0 1.]
L1 = Layer(W1, b, ReLU())
L2 = Layer(W2, b, Id())
full_net = Network([L1; L2])
# input range
x = [normal(interval(0,1),1), normal(interval(0,1),1)]

# Moment-propagation approach

Implementation of the moment propagation approach introduced in https://arxiv.org/abs/2403.16163. 

In [ ]:
function propagate_network(mean_x, cov_x, network::Network)
    for layer in network.layers
        mean_x, cov_x = propagate_layer(mean_x, cov_x, layer)
    end
    return mean_x, cov_x
end

In [ ]:
function propagate_layer(mean_x, cov_x, layer::Layer)
    mean_y = layer.weights * mean_x .+ layer.bias
    cov_y = layer.weights * cov_x * transpose(layer.weights)
    if layer.activation === ReLU()
        mean_z, cov_z = propagate_relu(mean_y, cov_y)
    elseif layer.activation === Id()
        mean_z, cov_z = mean_y, cov_y
    end
    
    return mean_z, cov_z
end

In [ ]:
function propagate_relu(mean_y, cov_y)
    var_y = diag(cov_y)
    std_y = sqrt.(var_y)

    std_y = std_y .+ 1e-8

    alpha = mean_y ./ std_y
    CDF_alpha = 0.5 * (1 .+ erf.(alpha ./ sqrt(2))) 
    PDF_alpha = exp.(-0.5 .* alpha.^2) ./ sqrt(2π) 

    mean_z = mean_y .* CDF_alpha .+ std_y .* PDF_alpha

    derelu = []
    push!(derelu,CDF_alpha)
    push!(derelu, PDF_alpha ./ std_y)
    push!(derelu, -mean_y ./ (std_y.^3) .* PDF_alpha)
    push!(derelu, PDF_alpha ./ (std_y.^3) .* ((mean_y.^2 ./ (std_y.^2)) .- 1))
    push!(derelu, - mean_y./ (std_y.^5) .* ((mean_y./std_y).^2 .- 3) .* PDF_alpha)

    n = length(mean_y)
    cov_z = zeros(n, n)
    for i in 1:n
        for j in 1:n
            rho_ij = cov_y[i, j] / (std_y[i] * std_y[j])
            for k in 1:5
                cov_z[i, j] += ((rho_ij^k)/factorial(k)) * (std_y[i]^k * derelu[k][i]) * (std_y[j]^k * derelu[k][j])
            end
        end
    end

    return mean_z, cov_z
end

Testing this on the toy example:

In [ ]:
nb_discretization_steps = 100
ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
pz = pbox_approximate_nnet(full_net, x, true)
ProbabilityBoundsAnalysis.plot(pz[1])

mean_arr = [0:0.01:1;]

for i in 1:length(mean_arr)
    for j in i:length(mean_arr)
        mean_input = [mean_arr[i], mean_arr[j]]
        cov_input = [1.0 0; 0 1.0]
        mean_output, cov_output = propagate_network(mean_input, cov_input, full_net)
        normal_dist = Normal(mean_output[1], cov_output[1,1])
        supp = range(mean_output[1] - 10, mean_output[1] + 10, length=1000)
        cdf_values = cdf.(normal_dist, supp)
        plot(supp, cdf_values)
    end
end
PyPlot.savefig("MomentPropagationfst.png")


In [ ]:
nb_discretization_steps = 100
ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
pz = pbox_approximate_nnet(full_net, x, true)
ProbabilityBoundsAnalysis.plot(pz[2])

mean_arr = [0:0.01:1;]

for i in 1:length(mean_arr)
    for j in i:length(mean_arr)
        mean_input = [mean_arr[i], mean_arr[j]]
        cov_input = [1.0 0; 0 1.0]
        mean_output, cov_output = propagate_network(mean_input, cov_input, full_net)
        normal_dist = Normal(mean_output[2], cov_output[2,2])
        supp = range(mean_output[2] - 20, mean_output[2] + 20, length=1000)
        cdf_values = cdf.(normal_dist, supp)
        plot(supp, cdf_values)
    end
end
PyPlot.savefig("MomentPropagationsnd.png")
